# Deploy Model to Red Hat OpenShift Data Science

This notebook deploys the model.joblib file to OpenShift Data Science through Inference Service.
It is required that the model.joblib and inference-service.yaml file exists.
In the pipeline, the model.joblib file is generated by the preceding job.

In [ ]:
import os, time, subprocess
from jinja2 import Template

variables = {}
with open("vars.txt") as myfile:
    for line in myfile:
        name, var = line.partition("=")[::2]
        variables[name.strip()] = str(var)

In [ ]:
template_data = {"model_version": variables['model_version']}
template = Template(open("inference-service.yaml").read())
rendered_template = template.render(template_data)
print('Rendered Template: \r' + rendered_template)

In [ ]:
subprocess.run(['oc', 'whoami'])
ps = subprocess.Popen(['echo', rendered_template], stdout=subprocess.PIPE)
output = subprocess.check_output(['oc', 'apply', '-f', '-'], stdin=ps.stdout)
ps.wait()
print(output)
print('Model deployment completed. Version: ' + variables['model_version'])